### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Arabic
Lang Id = 9
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U,https://www.youtube.com/watch?v=rVkXdGORB8U&t=...
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
...,...,...,...,...,...,...
195,أنا هنا,2930,2932,أنا هنا,cBzFipQTFs8,https://www.youtube.com/watch?v=cBzFipQTFs8&t=...
196,نعم أنا,3523,3525,نعم أنا,MNqYWTc6aRs,https://www.youtube.com/watch?v=MNqYWTc6aRs&t=...
197,من هناك,69,70,من هناك,miLnrq9ucaA,https://www.youtube.com/watch?v=miLnrq9ucaA&t=69s
198,كان ذلك,4571,4573,كان ذلك,9Wl7PoOwz6o,https://www.youtube.com/watch?v=9Wl7PoOwz6o&t=...


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,https://www.youtube.com/watch?v=NCem4xje9Ss&t=...
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,https://www.youtube.com/watch?v=HnSK6tiiQZg&t=...
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I,https://www.youtube.com/watch?v=VSx7oL5PC7I&t=...
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4,https://www.youtube.com/watch?v=gqC0SbiQQc4&t=...
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE,https://www.youtube.com/watch?v=gkhlMGmoTWE&t=...
...,...,...,...,...,...,...
195,و لم يكن لدينا وقت من أجل,353,355,و لم يكن لدينا وقت من أجل,g5PYbmQ8ZBg,https://www.youtube.com/watch?v=g5PYbmQ8ZBg&t=...
196,لا يجب أن يكون الأمر كذلك,555,558,لا يجب أن يكون الأمر كذلك,z1w0ZVrzZjw,https://www.youtube.com/watch?v=z1w0ZVrzZjw&t=...
197,إذا من الرجل الذي كان على,1306,1308,إذا من الرجل الذي كان على الهاوية,nyfxotxub_U,https://www.youtube.com/watch?v=nyfxotxub_U&t=...
198,أن هناك شيء ما قد حدث لها,879,881,أن هناك شيء ما قد حدث لها,MNqYWTc6aRs,https://www.youtube.com/watch?v=MNqYWTc6aRs&t=...


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,https://www.youtube.com/watch?v=FF1qfIe0Mb0&t=21s
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg,https://www.youtube.com/watch?v=Q4PgAY_qXKg&t=...
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM,https://www.youtube.com/watch?v=SJ0tF6gA0cM&t=...
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU,https://www.youtube.com/watch?v=lhfkYiCpTiU&t=...
...,...,...,...,...,...,...
195,أليس كذلك هذا هو وضع الحياة و الموت هنا,3804,3807,ركز أليس كذلك هذا هو وضع الحياة و الموت هنا,mzhP0xXlo1U,https://www.youtube.com/watch?v=mzhP0xXlo1U&t=...
196,في كل لعبة وفي كل عام كان هناك شيء جديد,224,229,في كل لعبة وفي كل عام كان هناك شيء جديد نتعلمه,07z2065eAtg,https://www.youtube.com/watch?v=07z2065eAtg&t=...
197,كنت أنا أي واحد منا ماذا يمكننا أن نفعل,1854,1857,سواء كنت أنا أي واحد منا ماذا يمكننا أن نفعل,UnN4H4zt3Po,https://www.youtube.com/watch?v=UnN4H4zt3Po&t=...
198,مرحبا يا صديق إلى أين أنت ذاهب هل تحتاج,5206,5208,مرحبا يا صديق إلى أين أنت ذاهب هل تحتاج لتوصيلة,r1ZCh9ZdOak,https://www.youtube.com/watch?v=r1ZCh9ZdOak&t=...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,سواء كنت تعتقد أنك تستطيع ذلك أو إذا كنت تعتقد...,39,54,قال هنري فورد سواء كنت تعتقد أنك تستطيع ذلك أو...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=39s
1,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,277,288,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,NbA_c5PoYMo,https://www.youtube.com/watch?v=NbA_c5PoYMo&t=...
2,من التفكير سأكون سعيدا عندما أكون في مكان مختل...,121,133,وبدلا من التفكير سأكون سعيدا عندما أكون في مكا...,nvEvYgP8gWM,https://www.youtube.com/watch?v=nvEvYgP8gWM&t=...
3,فيها ثلاثة أو أربعة أشياء التي يجب أن أقوم بها...,330,336,لا أتجاوز فيها ثلاثة أو أربعة أشياء التي يجب أ...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
4,شيء وهو مهم جدا وهو أن لا تحاول تغيير الشخص ال...,134,147,ثاني شيء وهو مهم جدا وهو أن لا تحاول تغيير الش...,7ub86ZM9Z3k,https://www.youtube.com/watch?v=7ub86ZM9Z3k&t=...
...,...,...,...,...,...,...
195,بعدها سنبدأ في جولة لمعرفة كيفية البدء في كتاب...,16,24,بعدها سنبدأ في جولة لمعرفة كيفية البدء في كتاب...,Ii5YIbltUh8,https://www.youtube.com/watch?v=Ii5YIbltUh8&t=16s
196,أنا أتساءل عنه أيضا على الرغم من أنه يرتدي هذا...,840,847,أنا أتساءل عنه أيضا على الرغم من أنه يرتدي هذا...,Ujl3rNsw8zM,https://www.youtube.com/watch?v=Ujl3rNsw8zM&t=...
197,هذه الآلة رائعة لمن هو في عجلة من أمره في الصب...,163,169,هذه الآلة رائعة لمن هو في عجلة من أمره في الصب...,YbSObFMCNOs,https://www.youtube.com/watch?v=YbSObFMCNOs&t=...
198,هو استخدم الآلة لفترة أيضا وبعد ذلك سنتحدث عن ...,556,559,هو استخدم الآلة لفترة أيضا وبعد ذلك سنتحدث عن ...,h_GDl4sr7Tw,https://www.youtube.com/watch?v=h_GDl4sr7Tw&t=...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,245,253,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,pkIhkqWKMm8,https://www.youtube.com/watch?v=pkIhkqWKMm8&t=...
1,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,221,229,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
2,لا تعيد منذ البداية و ربما تضيع فيه وقتا قد تق...,676,683,فكي لا تعيد منذ البداية و ربما تضيع فيه وقتا ق...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=...
3,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,148,161,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,WcKiIL31tqY,https://www.youtube.com/watch?v=WcKiIL31tqY&t=...
4,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,2920,2930,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,DlqJjdrYSi0,https://www.youtube.com/watch?v=DlqJjdrYSi0&t=...
...,...,...,...,...,...,...
195,و هذا لأننا قمنا بتغيير غرفة المعيشة لازال هنا...,52,57,و هذا لأننا قمنا بتغيير غرفة المعيشة لازال هنا...,hx18ZfXn2kQ,https://www.youtube.com/watch?v=hx18ZfXn2kQ&t=52s
196,أردت طوال الفيلم أن أشاهد كيف ستكون النهاية لأ...,1039,1044,أردت طوال الفيلم أن أشاهد كيف ستكون النهاية لأ...,_aoN8P7_1Ac,https://www.youtube.com/watch?v=_aoN8P7_1Ac&t=...
197,وأنت قلت مشهد القتال مع التنين تقريبا أنا أعتق...,1094,1101,وأنت قلت مشهد القتال مع التنين تقريبا أنا أعتق...,ONeLUP_URYA,https://www.youtube.com/watch?v=ONeLUP_URYA&t=...
198,كم كان عمرك عندما وجدت شخصية ترين نفسك فيها وم...,92,99,كم كان عمرك عندما وجدت شخصية ترين نفسك فيها وم...,r6fAt6TryQE,https://www.youtube.com/watch?v=r6fAt6TryQE&t=92s


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,238,253,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,iFKU6f1vXnI,https://www.youtube.com/watch?v=iFKU6f1vXnI&t=...
1,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,403,417,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,QinTLIO8P6A,https://www.youtube.com/watch?v=QinTLIO8P6A&t=...
2,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,333,352,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,I789EC6uRI4,https://www.youtube.com/watch?v=I789EC6uRI4&t=...
3,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,170,187,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,U9s_ompNZ9c,https://www.youtube.com/watch?v=U9s_ompNZ9c&t=...
4,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,69,86,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=69s
...,...,...,...,...,...,...
195,مؤخرا وجدت لي العديد من الصور أمام الحاسوب وأن...,65,74,مؤخرا وجدت لي العديد من الصور أمام الحاسوب وأن...,T8qVV0MZqV8,https://www.youtube.com/watch?v=T8qVV0MZqV8&t=65s
196,هذا ليس من شأنك أيها الصغير اللعين لدي شئ جيد ...,2174,2183,هذا ليس من شأنك أيها الصغير اللعين لدي شئ جيد ...,oOOotm88omw,https://www.youtube.com/watch?v=oOOotm88omw&t=...
197,لا تقلق بشأن حذف أي شيء تحتاجه فهذه مجرد كل ال...,340,347,لا تقلق بشأن حذف أي شيء تحتاجه فهذه مجرد كل ال...,2eMruOT5MIY,https://www.youtube.com/watch?v=2eMruOT5MIY&t=...
198,و لكن عندما يتعلق الأمر بالحديث فهو غير متأكد ...,85,94,و لكن عندما يتعلق الأمر بالحديث فهو غير متأكد ...,Fm213BhZgsU,https://www.youtube.com/watch?v=Fm213BhZgsU&t=85s


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4,https://www.youtube.com/watch?v=_bfxALYv4A4&t=...
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=9s
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI,https://www.youtube.com/watch?v=hVg78Xb6KlI&t=...
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE,https://www.youtube.com/watch?v=DiM0xqXXFcE&t=...
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA,https://www.youtube.com/watch?v=PX6Df4I95wA&t=...
...,...,...,...,...,...,...
195,سوف نتكلم اليوم عن نموذج الأعمال التجاري كي نر...,25,36,سوف نتكلم اليوم عن نموذج الأعمال التجاري كي نر...,2DxS2q_HSbM,https://www.youtube.com/watch?v=2DxS2q_HSbM&t=25s
196,اما الشكل الخارجى للسيارة فهو ممتاز جدا لوحة ف...,117,142,اما الشكل الخارجى للسيارة فهو ممتاز جدا لوحة ف...,TOy_KLnD2qM,https://www.youtube.com/watch?v=TOy_KLnD2qM&t=...
197,أنا أتتبع موقعك ولكن لا يبدو أنني أستطيع الوصو...,622,629,أيزك أنا أتتبع موقعك ولكن لا يبدو أنني أستطيع ...,Nuqx3eCUQI0,https://www.youtube.com/watch?v=Nuqx3eCUQI0&t=...
198,قد لا يستطيع المريض التنفس تماما و لو لم نستطي...,297,307,قد لا يستطيع المريض التنفس تماما و لو لم نستطي...,Mmr80C7sHBY,https://www.youtube.com/watch?v=Mmr80C7sHBY&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1676968569
  nanos: 54112000
}